<b>Background


Mumbai is the largest city in India with more than 20 million resident. Mumbai city attracks talent from all over the country. People can find lot of good employment options in Mumbai.

However, Mumbai is also very large city. It is desirable to stay in the locality so that commute to office is easy. if right area is not selected then it may take 4-5 hours just to commute to office one way. Sure, one can forget about the work then.

There are multiple parameters one will like to evaluate before buying/leasing a property in a particular area. If I am currently staying in the Mumbai, I may want to find the property in the locality which is similar to my current locality. It may be possible, I want to live in the locality which provides venues which are similar to most posh localities of the city. However, Budget/affordability also plays a key role here.

Target Audience - This project will be useful to the people who are looking for buying/renting proprty in Mumbai. Using this project, one can find similar localities as per thier current standards or aspirational standards. I have also put in price as a parameter for comparison so that they can finalize locality based on thier budget.



In [15]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

<b> Download and Explore Data

We will need 3 sets of data to complete this project
1. Locality data and housing prices data
2. Locality lat long data
3. locality venues data

I am planning to extract housing pricing data available at Magicbricks website. Magicbricks is one of the popular properly listing site in the India. I am planning to use mapsquare tool for lat long data and I am planning to use Foursquare API to get venue details.

I have compiled an csv file using data available on https://www.magicbricks.com/Property-Rates-Trends/ALL-RESIDENTIAL-rates-in-Mumbai and have added lat long using MapSquare API

I have added the file on Google Drive  - https://drive.google.com/file/d/178NGiwTy-mlibskihi0fOtaOPFe4IXgY/view?usp=sharing and IBM server

I am going to extract data from IBM Drive using this code.

In [10]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

client_d5d9 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='zHW3vWjFUlaiMClDzlq5SYQeB8AGWp6zG5SiDD-kBeeM',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_d5d9.get_object(Bucket='capstoneproject-donotdelete-pr-131jtxibkrupec',Key='Book1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_Mum = pd.read_csv(body)
df_Mum.head()


,Locality,Price,Latitude,Longitude
0,Aarey Milk Colony,13504,19.156129,72.870722
1,Agripada,26788,18.975302,72.824897
2,Alika Nagar,14228,19.198397,72.874267
3,Altamount Road,71016,18.966362,72.809148
4,Amboli,20908,19.127587,72.847115


In [11]:
df_Mum.shape

(116, 4)

In [12]:
df_Mum.dtypes

Locality      object
Price          int64
Latitude     float64
Longitude    float64
dtype: object

Exploring Inputted data on the map

In [13]:
address = 'Mumbai, IN'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai are 18.9387711, 72.8353355.


In [16]:
map_mumbai = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_Mum['Latitude'], df_Mum['Longitude'], df_Mum['Locality']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mumbai)  
    
map_mumbai

Now Importing Neighbourhood data from Foursqure

In [17]:
CLIENT_ID = '0KN5HVMTQ10BUNLL5EM3FDCLZV33MS4VRCW4JAIHJSA2PDWG' # your Foursquare ID
CLIENT_SECRET = 'VDOA4E5UXWY1WC3PEIM33TPPV3FGK542QJT2DXNOLBFPSVKA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0KN5HVMTQ10BUNLL5EM3FDCLZV33MS4VRCW4JAIHJSA2PDWG
CLIENT_SECRET:VDOA4E5UXWY1WC3PEIM33TPPV3FGK542QJT2DXNOLBFPSVKA


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=750):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Locality', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
LIMIT = 100
Mumbai_venues = getNearbyVenues(names=df_Mum['Locality'],
                                   latitudes=df_Mum['Latitude'],
                                   longitudes=df_Mum['Longitude']
                                  )

Aarey Milk Colony
Agripada
Alika Nagar
Altamount Road
Amboli
Anand Nagar
Andheri
Andheri East
Andheri West
Andheri-Goregaon
Asha Nagar
Ashok Nagar Central Mumbai
Ashok Van
Azad Nagar 2
Azad Nagar Versova Road
Bandra
Bandra East
Bandra Kurla Complex
Bandra West
Bangur Nagar
Bhakti Park
Bhandup
Bhandup East
Bhandup West
Bhayandar
Bhayandar East
Bhayandar West
Bhoiwada
Borivali
Borivali East
Borivali West
Breach Candy
Byculla
Central Avenue Road
Chakala
Chandivali
Charkop
Charkop Sector 2
Charkop Sector 8
Chembur
Chembur Colony
Colaba
Cuffe Parade
D.N. Nagar
Dadar
Dadar East
Dadar TT Circle
Dadar West
Dahisar
Dahisar East
Dahisar West
Deonar
Dindoshi
Dongarpada
Eksar
Evershine Nagar
Film City Road
Ghatkopar
Ghatkopar East
Ghatkopar West
Gokuldham
Goregaon
Goregaon East
Goregaon West
Govandi East
Guru Teg Bahadur Nagar
Hanuman Nagar
Vikhroli
Vikhroli East
Vikhroli West
Vile Parle
Vile Parle East
Vile Parle West
Vinay Nagar
Wadala
Wadala East
Wadala West
Walkeshwar
Worli
Worli Hill
Thane
Ud

In [24]:
Mumbai_venues.head(10)

,Locality,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aarey Milk Colony,19.156129,72.870722,cafe coffee day,19.159053,72.869031,Coffee Shop
1,Aarey Milk Colony,19.156129,72.870722,Panchvati Fast Food Corner,19.157628,72.874506,Fast Food Restaurant
2,Aarey Milk Colony,19.156129,72.870722,Chota Kashmir Boating,19.160905,72.872482,Lake
3,Aarey Milk Colony,19.156129,72.870722,Panchavati Fast Food Corner,19.157709,72.876511,Café
4,Aarey Milk Colony,19.156129,72.870722,Subway,19.158622,72.876653,Sandwich Place
5,Agripada,18.975302,72.824897,Celejor,18.975844,72.823679,Bakery
6,Agripada,18.975302,72.824897,Tote On The Turf,18.980266,72.820294,Nightclub
7,Agripada,18.975302,72.824897,cafe coffee day,18.976988,72.824051,Coffee Shop
8,Agripada,18.975302,72.824897,Neel,18.980407,72.820403,Indian Restaurant
9,Agripada,18.975302,72.824897,Maratha Mandir,18.971266,72.821819,Movie Theater


In [25]:
print('There are {} uniques categories.'.format(len(Mumbai_venues['Venue Category'].unique())))

There are 201 uniques categories.


Time to analyze each neigbourhood

In [26]:
Mumbai_d = pd.get_dummies(Mumbai_venues[['Venue Category']], prefix="", prefix_sep="")

Mumbai_d['zLocality'] = Mumbai_venues['Locality'] 

fixed_columns = [Mumbai_d.columns[-1]] + list(Mumbai_d.columns[:-1])
Mumbai_d = Mumbai_d[fixed_columns]

Mumbai_d.head()

,zLocality,Accessories Store,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Cafeteria,Café,Camera Store,Campground,Candy Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Cafeteria,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Electronics Store,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mountain,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Tea Room,Tennis Court,Thai Restaurant,Theater,Track,Track Stadium,Trail,Train,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Volleyball Court,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Aarey Milk Colony,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Aarey Milk Colony,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Aarey Milk Colony,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0

-------------------

Total count for each neighbourhood

In [28]:
Mumbai_venue_mean = Mumbai_d.groupby('zLocality').mean().reset_index()
Mumbai_venue_mean

,zLocality,Accessories Store,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Cafeteria,Café,Camera Store,Campground,Candy Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Cafeteria,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Electronics Store,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mountain,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Tea Room,Tennis Court,Thai Restaurant,Theater,Track,Track Stadium,Trail,Train,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Volleyball Court,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Aarey Milk Colony,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.2000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

Analyzing each locality in terms of most frequent venue

In [29]:
num_top_venues = 5

for hood in Mumbai_venue_mean['zLocality']:
    print("----"+hood+"----")
    temp = Mumbai_venue_mean[Mumbai_venue_mean['zLocality'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aarey Milk Colony----
                  venue  freq
0           Coffee Shop   0.2
1                  Café   0.2
2        Sandwich Place   0.2
3                  Lake   0.2
4  Fast Food Restaurant   0.2


----Agripada----
               venue  freq
0         Restaurant  0.13
1  Indian Restaurant  0.13
2                Gym  0.13
3             Bakery  0.13
4        Coffee Shop  0.07


----Alika Nagar----
               venue  freq
0  Indian Restaurant  0.13
1        Coffee Shop  0.13
2     Ice Cream Shop  0.07
3             Bakery  0.07
4              Plaza  0.07


----Altamount Road----
            venue  freq
0            Café  0.09
1          Bakery  0.07
2     Pizza Place  0.05
3     Coffee Shop  0.05
4  Sandwich Place  0.05


----Amboli----
                venue  freq
0         Coffee Shop  0.29
1   Indian Restaurant  0.29
2         Pizza Place  0.14
3      Sandwich Place  0.14
4  Chinese Restaurant  0.14


----Anand Nagar----
                           venue  freq
0             

Lets see top 10 popular joints for each place and we will use these to understand popular localities which does not have mexican joints

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Locality']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Locality'] = Mumbai_venue_mean['zLocality']

for ind in np.arange(Mumbai_venue_mean.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Mumbai_venue_mean.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Locality,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Aarey Milk Colony,Sandwich Place,Fast Food Restaurant,Lake,Café,Coffee Shop,Zoo,Factory,Flea Market,Fish Market,Field,Farmers Market,Farm,Falafel Restaurant,Event Space,Food,Electronics Store,Donut Shop,Dive Bar,Diner,Dim Sum Restaurant
1,Agripada,Indian Restaurant,Bakery,Gym,Restaurant,Japanese Restaurant,Coffee Shop,Nightclub,Movie Theater,Fast Food Restaurant,Mediterranean Restaurant,Bank,Zoo,Farm,Farmers Market,Factory,Field,Fish Market,Flea Market,Falafel Restaurant,Electronics Store
2,Alika Nagar,Indian Restaurant,Coffee Shop,Sandwich Place,Shopping Mall,Gym / Fitness Center,Dessert Shop,Arts & Crafts Store,Food Truck,Ice Cream Shop,Chinese Restaurant,Soccer Field,Bakery,Plaza,Farm,Zoo,Farmers Market,Fast Food Restaurant,Field,Fish Market,Falafel Restaurant
3,Altamount Road,Café,Bakery,Sandwich Place,Coffee Shop,Pizza Place,Salon / Barbershop,Indian Restaurant,Hotel,Chinese Restaurant,Snack Place,Department Store,Lounge,Brewery,Italian Restaurant,Bookstore,Japanese Restaurant,Dessert Shop,Restaurant,Racetrack,Falafel Restaurant
4,Amboli,Coffee Shop,Indian Restaurant,Sandwich Place,Pizza Place,Chinese Restaurant,Event Space,Flea Market,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Zoo,Electronics Store,Food,Donut Shop,Dive Bar,Diner


Using machine learning to identify similar localities

In [33]:
# set number of clusters
kclusters = 5

Mumbai_grouped_clustering = Mumbai_venue_mean.drop('zLocality', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Mumbai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 3, 1, 1, 3, 1, 3, 1, 3, 3], dtype=int32)

In [35]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [36]:
Mumbai_combined= df_Mum
Mumbai_combined = Mumbai_combined.join(neighborhoods_venues_sorted.set_index('Locality'), on='Locality')
Mumbai_combined.dropna(axis=0,how='any',inplace=True)
Mumbai_combined.head()

,Locality,Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Aarey Milk Colony,13504,19.156129,72.870722,1,Sandwich Place,Fast Food Restaurant,Lake,Café,Coffee Shop,Zoo,Factory,Flea Market,Fish Market,Field,Farmers Market,Farm,Falafel Restaurant,Event Space,Food,Electronics Store,Donut Shop,Dive Bar,Diner,Dim Sum Restaurant
1,Agripada,26788,18.975302,72.824897,3,Indian Restaurant,Bakery,Gym,Restaurant,Japanese Restaurant,Coffee Shop,Nightclub,Movie Theater,Fast Food Restaurant,Mediterranean Restaurant,Bank,Zoo,Farm,Farmers Market,Factory,Field,Fish Market,Flea Market,Falafel Restaurant,Electronics Store
2,Alika Nagar,14228,19.198397,72.874267,1,Indian Restaurant,Coffee Shop,Sandwich Place,Shopping Mall,Gym / Fitness Center,Dessert Shop,Arts & Crafts Store,Food Truck,Ice Cream Shop,Chinese Restaurant,Soccer Field,Bakery,Plaza,Farm,Zoo,Farmers Market,Fast Food Restaurant,Field,Fish Market,Falafel Restaurant
3,Altamount Road,71016,18.966362,72.809148,1,Café,Bakery,Sandwich Place,Coffee Shop,Pizza Place,Salon / Barbershop,Indian Restaurant,Hotel,Chinese Restaurant,Snack Place,Department Store,Lounge,Brewery,Italian Restaurant,Bookstore,Japanese Restaurant,Dessert Shop,Restaurant,Racetrack,Falafel Restaurant
4,Amboli,20908,19.127587,72.847115,3,Coffee Shop,Indian Restaurant,Sandwich Place,Pizza Place,Chinese Restaurant,Event Space,Flea Market,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Zoo,Electronics Store,Food,Donut Shop,Dive Bar,Diner


<b>Visualizing the results

In [39]:
#create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Mumbai_combined['Latitude'], Mumbai_combined['Longitude'], Mumbai_combined['Locality'], Mumbai_combined['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Analyzing each Cluster for price and similarity

In [45]:
cluster1 = Mumbai_combined.loc[Mumbai_combined['Cluster Labels'] == 0, Mumbai_combined.columns[[0] + list(range(1, Mumbai_combined.shape[1]))]]
cluster1.head(10)

,Locality,Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
12,Ashok Van,13798,19.063103,72.912934,0,Fried Chicken Joint,Restaurant,Zoo,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Event Space,Electronics Store,Donut Shop,Dive Bar,Diner,Dim Sum Restaurant,Dessert Shop
56,Film City Road,15827,19.151023,72.891437,0,Resort,Gym / Fitness Center,Restaurant,Golf Course,Zoo,Electronics Store,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Event Space,Donut Shop,Flower Shop,Dive Bar,Diner,Dim Sum Restaurant,Dessert Shop
87,Powai lake,20030,19.127662,72.906332,0,Restaurant,Café,Indian Restaurant,Asian Restaurant,Zoo,Factory,Flower Shop,Flea Market,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Event Space,Food & Drink Shop,Donut Shop,Dive Bar,Diner
95,Saki Vihar Road,17500,19.128608,72.895732,0,Restaurant,Lounge,Hotel,Indian Restaurant,Garden,Chinese Restaurant,Tea Room,Zoo,Factory,Flea Market,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Food,Electronics Store,Donut Shop


In [46]:
cluster2 = Mumbai_combined.loc[Mumbai_combined['Cluster Labels'] == 1, Mumbai_combined.columns[[0] + list(range(1, Mumbai_combined.shape[1]))]]
cluster2.head(10)

,Locality,Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Aarey Milk Colony,13504,19.156129,72.870722,1,Sandwich Place,Fast Food Restaurant,Lake,Café,Coffee Shop,Zoo,Factory,Flea Market,Fish Market,Field,Farmers Market,Farm,Falafel Restaurant,Event Space,Food,Electronics Store,Donut Shop,Dive Bar,Diner,Dim Sum Restaurant
2,Alika Nagar,14228,19.198397,72.874267,1,Indian Restaurant,Coffee Shop,Sandwich Place,Shopping Mall,Gym / Fitness Center,Dessert Shop,Arts & Crafts Store,Food Truck,Ice Cream Shop,Chinese Restaurant,Soccer Field,Bakery,Plaza,Farm,Zoo,Farmers Market,Fast Food Restaurant,Field,Fish Market,Falafel Restaurant
3,Altamount Road,71016,18.966362,72.809148,1,Café,Bakery,Sandwich Place,Coffee Shop,Pizza Place,Salon / Barbershop,Indian Restaurant,Hotel,Chinese Restaurant,Snack Place,Department Store,Lounge,Brewery,Italian Restaurant,Bookstore,Japanese Restaurant,Dessert Shop,Restaurant,Racetrack,Falafel Restaurant
5,Anand Nagar,13990,18.966523,72.811888,1,Chinese Restaurant,Café,Pizza Place,Coffee Shop,Bakery,Vegetarian / Vegan Restaurant,Concert Hall,Fast Food Restaurant,Snack Place,Bookstore,Food Truck,Deli / Bodega,Sandwich Place,Salon / Barbershop,Brewery,Restaurant,Lounge,Farmers Market,Falafel Restaurant,Cafeteria
7,Andheri East,18033,19.115883,72.854202,1,Indian Restaurant,Hotel,Ice Cream Shop,Chinese Restaurant,Vegetarian / Vegan Restaurant,Sandwich Place,Cocktail Bar,Burger Joint,Food Truck,Pizza Place,Camera Store,Lounge,Café,Shopping Mall,Fish Market,Multiplex,Bus Station,Juice Bar,Farmers Market,Field
10,Asha Nagar,16495,19.210955,72.864322,1,Lounge,Movie Theater,Restaurant,Seafood Restaurant,Sporting Goods Shop,Food & Drink Shop,Snack Place,Skating Rink,Sandwich Place,Burger Joint,Food Truck,Café,Campground,Men's Store,Clothing Store,Pizza Place,Pharmacy,Sports Bar,Neighborhood,Fried Chicken Joint
11,Ashok Nagar Central Mumbai,14841,19.052995,72.879788,1,Café,Electronics Store,Performing Arts Venue,Soccer Field,Vegetarian / Vegan Restaurant,Food,Hotel,Indian Restaurant,Zoo,Farm,Farmers Market,Field,Fast Food Restaurant,Factory,Fish Market,Flea Market,Falafel Restaurant,Donut Shop,Event Space,Dive Bar
13,Azad Nagar 2,21730,19.092060,72.899401,1,Café,Food Truck,Factory,Indian Restaurant,Donut Shop,Food,Flower Shop,Flea Market,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Food Court,Electronics Store,Dive Bar,Diner,Dim Sum Restaurant
14,Azad Nagar Versova Road,20355,19.126910,72.837648,1,Bar,Indian Restaurant,Pub,Chinese Restaurant,Coffee Shop,Pizza Place,Park,Burger Joint,Sandwich Place,Bowling Alley,Smoke Shop,Snack Place,Cocktail Bar,Falafel Restaurant,College Cafeteria,Ice Cream Shop,Gym,Athletics & Sports,Asian Restaurant,Vegetarian / Vegan Restaurant
16,Bandra East,31525,19.061514,72.850093,1,Indian Restaurant,Restaurant,Café,Pizza Place,Fast Food Restaurant,Spa,Bar,French Restaurant,Bistro,Chinese Restaurant,Sports Club,Brazilian Restaurant,Multiplex,Gym,Seafood Restaurant,Field,Farmers Market,Farm,Zoo,Event Space


In [47]:
cluster3 = Mumbai_combined.loc[Mumbai_combined['Cluster Labels'] == 2, Mumbai_combined.columns[[0] + list(range(1, Mumbai_combined.shape[1]))]]
cluster3.head(10)

,Locality,Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
20,Bhakti Park,21823,19.027868,72.879568,2,Coffee Shop,Bookstore,Thai Restaurant,Multiplex,Light Rail Station,Asian Restaurant,Snack Place,Gym Pool,Baseball Field,Fish Market,Factory,Field,Fast Food Restaurant,Farmers Market,Flea Market,Farm,Falafel Restaurant,Zoo,Event Space,Food
38,Charkop Sector 8,13312,19.216097,72.814769,2,Snack Place,Scenic Lookout,Coffee Shop,Smoke Shop,Event Space,Flower Shop,Flea Market,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Electronics Store,Donut Shop,Dive Bar,Diner,Dim Sum Restaurant,Dessert Shop
74,Wadala,21705,19.026919,72.875934,2,Coffee Shop,Bookstore,Thai Restaurant,Multiplex,Light Rail Station,Asian Restaurant,Snack Place,Gym Pool,Baseball Field,Fish Market,Factory,Field,Fast Food Restaurant,Farmers Market,Flea Market,Farm,Falafel Restaurant,Zoo,Event Space,Food
75,Wadala East,21651,19.026919,72.875934,2,Coffee Shop,Bookstore,Thai Restaurant,Multiplex,Light Rail Station,Asian Restaurant,Snack Place,Gym Pool,Baseball Field,Fish Market,Factory,Field,Fast Food Restaurant,Farmers Market,Flea Market,Farm,Falafel Restaurant,Zoo,Event Space,Food
76,Wadala West,24504,19.026919,72.875934,2,Coffee Shop,Bookstore,Thai Restaurant,Multiplex,Light Rail Station,Asian Restaurant,Snack Place,Gym Pool,Baseball Field,Fish Market,Factory,Field,Fast Food Restaurant,Farmers Market,Flea Market,Farm,Falafel Restaurant,Zoo,Event Space,Food


In [48]:
cluster4 = Mumbai_combined.loc[Mumbai_combined['Cluster Labels'] == 3, Mumbai_combined.columns[[0] + list(range(1, Mumbai_combined.shape[1]))]]
cluster4.head(10)

,Locality,Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
1,Agripada,26788,18.975302,72.824897,3,Indian Restaurant,Bakery,Gym,Restaurant,Japanese Restaurant,Coffee Shop,Nightclub,Movie Theater,Fast Food Restaurant,Mediterranean Restaurant,Bank,Zoo,Farm,Farmers Market,Factory,Field,Fish Market,Flea Market,Falafel Restaurant,Electronics Store
4,Amboli,20908,19.127587,72.847115,3,Coffee Shop,Indian Restaurant,Sandwich Place,Pizza Place,Chinese Restaurant,Event Space,Flea Market,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Zoo,Electronics Store,Food,Donut Shop,Dive Bar,Diner
6,Andheri,21327,19.120371,72.848043,3,Indian Restaurant,Ice Cream Shop,Bakery,Food,Sandwich Place,Food Court,Lounge,Burger Joint,Pizza Place,Fast Food Restaurant,Restaurant,Farmers Market,Zoo,Field,Falafel Restaurant,Fish Market,Farm,Electronics Store,Factory,Event Space
8,Andheri West,23465,19.117249,72.833968,3,Indian Restaurant,Gym / Fitness Center,Bar,Multiplex,Coffee Shop,Pub,Café,Smoke Shop,Snack Place,Lounge,Diner,Nightclub,Clothing Store,Park,Vegetarian / Vegan Restaurant,Bagel Shop,Women's Store,Deli / Bodega,Factory,Flea Market
9,Andheri-Goregaon,18889,19.187165,72.858398,3,Department Store,Gym / Fitness Center,Indian Restaurant,Zoo,Factory,Food,Flower Shop,Flea Market,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Event Space,Food Court,Donut Shop,Dive Bar,Diner
15,Bandra,34105,19.054979,72.840220,3,Indian Restaurant,Seafood Restaurant,Café,Pub,Bar,Korean Restaurant,Shopping Mall,Men's Store,Snack Place,Italian Restaurant,Clothing Store,Indie Movie Theater,Pizza Place,Restaurant,Steakhouse,Coffee Shop,BBQ Joint,Convenience Store,Train Station,Lounge
21,Bhandup,14771,19.143374,72.938350,3,Indian Restaurant,Asian Restaurant,Shopping Mall,Fast Food Restaurant,Big Box Store,Arcade,Dessert Shop,Train Station,Café,Bookstore,Pizza Place,Fish Market,Field,Zoo,Farm,Flea Market,Flower Shop,Farmers Market,Electronics Store,Falafel Restaurant
22,Bhandup East,14878,19.148557,72.947066,3,Train Station,Indian Restaurant,Ice Cream Shop,Asian Restaurant,Fast Food Restaurant,Bakery,Zoo,Factory,Food,Flower Shop,Flea Market,Fish Market,Field,Farmers Market,Farm,Falafel Restaurant,Donut Shop,Event Space,Electronics Store,Food Court
27,Bhoiwada,24945,18.953647,72.830238,3,Indian Restaurant,Dessert Shop,Ice Cream Shop,Café,Juice Bar,Fast Food Restaurant,Chinese Restaurant,Food,Indian Sweet Shop,Bakery,BBQ Joint,Convenience Store,Rest Area,Restaurant,Arcade,Antique Shop,Flea Market,Fish Market,Field,Flower Shop
29,Borivali East,15892,19.226723,72.861933,3,Indian Restaurant,Café,Snack Place,Garden Center,Scenic Lookout,Bike Rental / Bike Share,Pizza Place,Clothing Store,Burger Joint,Restaurant,Park,Department Store,Gym / Fitness Center,Bus Station,Chinese Restaurant,Fast Food Restaurant,Field,Farmers Market,Farm,Zoo


In [49]:
cluster5 = Mumbai_combined.loc[Mumbai_combined['Cluster Labels'] == 4, Mumbai_combined.columns[[0] + list(range(1, Mumbai_combined.shape[1]))]]
cluster5.head(10)

,Locality,Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
24,Bhayandar,9106,19.197152,72.811366,4,Resort,Dance Studio,Indian Restaurant,Diner,Zoo,Flower Shop,Flea Market,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Electronics Store,Event Space,Food & Drink Shop,Donut Shop,Dive Bar,Dim Sum Restaurant
25,Bhayandar East,8142,19.197152,72.811366,4,Resort,Dance Studio,Indian Restaurant,Diner,Zoo,Flower Shop,Flea Market,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Electronics Store,Event Space,Food & Drink Shop,Donut Shop,Dive Bar,Dim Sum Restaurant
26,Bhayandar West,9132,19.197152,72.811366,4,Resort,Dance Studio,Indian Restaurant,Diner,Zoo,Flower Shop,Flea Market,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Electronics Store,Event Space,Food & Drink Shop,Donut Shop,Dive Bar,Dim Sum Restaurant


Now based on current location of the user, user can find her/his area in the cluster and user can understand which area is almost similar to current locality and based on budget preference user can explore the locality.